In [1]:
import json
with open('contracts/WhiteList.compiled.json', 'r') as f:
    WhiteListOutput = json.loads(f.read())

In [3]:
WhiteListAbi = WhiteListOutput['contracts']['WhiteList.sol:WhiteList']['abi'] 
WhiteListBin = WhiteListOutput['contracts']['WhiteList.sol:WhiteList']['bin']


In [10]:
from web3 import Web3

w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:28545'))
w3.isConnected()
with open('keys.json', 'r') as f:
    private_keys = json.loads(f.read())['private_keys']
w3.eth.accounts

['0x300879aDf36B12a0EbB9de52E15EB59F3fF1574B',
 '0xf8C811F978bB623A8a37b6898b9A083b13Ef469E',
 '0x34A1e32dAF41E057131E47d30a53F402D070DdFd']

In [7]:
def sign_transaction(txn, account, private_key):
    next_nounce = w3.eth.getTransactionCount(account)
    
    signable_transaction = dict(
    txn,
    nonce=next_nounce,
    gasPrice=w3.toWei(4, 'gwei'))
    
    signature_info = w3.eth.account.signTransaction(signable_transaction, private_key)
    txn_hash = w3.eth.sendRawTransaction(signature_info.rawTransaction)
    receipt = w3.eth.waitForTransactionReceipt(txn_hash)
    return receipt

In [12]:
from web3.contract import ConciseContract

whitelist_deployer = w3.eth.contract(abi=WhiteListAbi, bytecode=WhiteListBin)
init = whitelist_deployer.constructor()
txn = init.buildTransaction({'gas': 1000000})
receipt = sign_transaction(txn, w3.eth.accounts[0], private_keys[w3.eth.accounts[0].lower()])
WhiteListAddress = receipt.contractAddress

# Build ConciseContract artifact
whitelist_contract = w3.eth.contract(
    address=WhiteListAddress,
    abi=WhiteListAbi,
)
whitelist_CContract = ConciseContract(whitelist_contract)
print("WhiteListAddress:             " + WhiteListAddress + "    Deployed:" + str(whitelist_CContract.isOwner()) )


WhiteListAddress:             0xaF7a33aDd1439598b3a897E60004331750132DBD    Deployed:True


In [13]:
# Whitelist 
insert_list = [w3.eth.accounts[1], w3.eth.accounts[2]]

# Submit txn
txn = whitelist_contract.functions.add(insert_list).buildTransaction({'gas': 320000})
receipt = sign_transaction(txn, w3.eth.accounts[0], private_keys[w3.eth.accounts[0].lower()])
receipt.status

1

In [15]:
# Using the standard web3 contract call 
dst_name = whitelist_contract.functions.isInWhiteList(w3.eth.accounts[1]).call()
dst_name

True

In [16]:
# Using the standard web3 contract call 
dst_name = whitelist_contract.functions.isInWhiteList(w3.eth.accounts[2]).call()
dst_name

True

In [17]:
# Using the standard web3 contract call 
dst_name = whitelist_contract.functions.isInWhiteList(w3.eth.accounts[0]).call()
dst_name

False